# Приложение для тех, кто любит читать

## Введение

Наша компания недавно купила крупный сервис для чтения книг по подписке.  
Задача: проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задачи:**
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — чтобы исключить из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': '--', # пароль
'host': '--',
'port': --, # порт подключения
'db': '--'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Книги

In [3]:
# выгрузим все данные
query = '''
            SELECT *
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine) 
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Авторы

In [4]:
# выгрузим все данные
query = '''
            SELECT *
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine) 
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


## Издательства

In [5]:
query = '''
            SELECT *
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine) 
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Рейтинги

In [6]:
query = '''
            SELECT *
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine) 
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


## Обзоры 

In [7]:
query = '''
            SELECT *
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine) 
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Запросы

### Cколько книг вышло после 1 января 2000 года?

In [8]:
query = '''
            SELECT COUNT(book_id)
            FROM books
            WHERE publication_date > '2000-01-01';
        '''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитать количество обзоров и среднюю оценку

In [9]:
query = '''
            SELECT b.title,
                   b.book_id,
                   COUNT(DISTINCT rv.review_id) as review_count,
                   ROUND(AVG(rating)) as avg_rating
            FROM books as b
            LEFT OUTER JOIN reviews as rv ON b.book_id = rv.book_id
            LEFT OUTER JOIN ratings as r ON b.book_id=r.book_id
            GROUP BY b.book_id
            ORDER BY review_count DESC, avg_rating DESC;
        '''
pd.io.sql.read_sql(query, con = engine)

,title,book_id,review_count,avg_rating
0,Twilight (Twilight #1),948,7,4.0
1,The Road,854,6,4.0
2,The Alchemist,627,6,4.0
3,The Giver (The Giver #1),733,6,4.0
4,The Catcher in the Rye,673,6,4.0
...,...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,191,0,4.0
996,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,4.0
997,Essential Tales and Poems,221,0,4.0
998,Leonardo's Notebooks,387,0,4.0


Мы получили описание всех книг с количеством обзоров и средней оценкой. Абсолютный победитель в обеих категориях - "Сумерки". 

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [10]:
query = '''
            SELECT publisher,
                   COUNT(b.book_id)
            FROM publishers AS p
            JOIN books AS b ON b.publisher_id=p.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY COUNT(b.book_id) DESC
            LIMIT 1;
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


Издательство Penguin Books	постаралось больше всех - 42 книги. Не удивительно - издательство с почти 100-летней историей внескло большой вклад в развитие книгопечатания в Британии.

### Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками

In [11]:
query = '''
            SELECT author,
                   AVG(rating)
            FROM authors AS a
            JOIN books AS b on b.author_id=a.author_id
            JOIN ratings AS r on r.book_id=b.book_id
            WHERE b.book_id IN (SELECT b.book_id
                                     FROM ratings AS r
                                     JOIN books AS b on b.book_id=r.book_id
                                     GROUP BY b.book_id
                                     HAVING COUNT(rating_id) > 50)
            GROUP BY author
            ORDER BY AVG(rating) DESC
            LIMIT 1;
        '''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Джоан Роулинг - самый любимый писатель современности.

### Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [13]:
query = '''
           SELECT AVG(reviews_count) AS avg_reviews
           FROM 
            (SELECT username,
                    COUNT(review_id) AS reviews_count
             FROM reviews
             WHERE username IN (SELECT username
                                FROM ratings
                                GROUP BY username
                                HAVING COUNT(rating) > 50)
             GROUP BY username) AS users
                    
           
        '''

pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.333333


Топовые обозреватели написали в среднем 24 обзора.

## Вывод

Был проведен небольшой анализ книжного рынка, при помощи SQL-запросов мы установили, что:
- после 1 января 2000 года было выпущено 819 книг, которые были доступны на нашем сервисе;
- по количеству обзоров и рейтингу лидируют книги, которые были экранизированы, абсолютный лидер в обеих категориях - "Сумерки";
- больше всего книг было выпущено издательством Penguin Books - это Британское издательство, работающее с 1935 года;
- Джоан Роулинг - автор с самой высокой средней оценкой книг;
- самые активные пользователи в среднем поставили 24 оценки.

У нас есть неплохая база, с помощью которой можно изучить поведние пользователей, чтобы предлагать им именно такие книги, какие они бы хотели читать.